In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests

In [3]:
!pip install bs4

  Preparing metadata (setup.py) ... - done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=c8155ad4d9a9a11cd202af9c77771b2767d28a387a23fabf0cccf777766732e1
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [4]:
from bs4 import BeautifulSoup

In [5]:
webpage = requests.get('https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page=1')     #{"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [6]:
webpage.text

'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;ambitionbox&#46;com&#47;list&#45;of&#45;companies&#63;" on this server.<P>\nReference&#32;&#35;18&#46;7cf06e68&#46;1700773860&#46;d1f9545\n</BODY>\n</HTML>\n'

In [7]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
webpage = requests.get('https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page=1', headers = headers).text

In [8]:
soup = BeautifulSoup(webpage, 'lxml')

In [9]:
#print(soup.prettify())

In [10]:
soup.find_all('h1')[0].text

'Access Denied'

In [11]:
len(soup.find_all('h2'))

0

In [12]:
for i in soup.find_all('h2'):
    print(i.text.strip())

In [13]:
for i in soup.find_all('div', class_ = 'companyCardWrapper__companyRating rating-35'):
    print(i.text.strip())

In [14]:
# for i in soup.find_all('span', class_ = "companyCardWrapper__ActionCount"):
#     print(i.text.strip())

In [15]:
company = soup.find_all('div', class_ = 'companyCardWrapper')

In [16]:
len(company)

0

In [17]:
names = []
ratings = []
reviews = []

ctype = []
employees = []
hq = []
old = []


for i in company:
    names.append(i.find('h2', class_ = 'companyCardWrapper__companyName').text.strip())
    ratings.append(i.find('span', class_ = 'companyCardWrapper__companyRatingValue').text.strip())
    reviews.append(i.find('span', class_ = 'companyCardWrapper__ActionCount').text.strip())
    k = i.find('span', class_ = 'companyCardWrapper__interLinking').text.strip().split(' | ')
    for j in range(len(k)):
        ctype.append(k[j])
        employees.append(k[j + 1])
        hq.append(k[j + 2])
        old.append(k[j + 3])
        break
    

In [18]:
ctype

[]

In [19]:
old

[]

In [20]:
d = {'names' : names, 'ratings' : ratings, 'reviews' : reviews, 'ctype' : ctype, 'employees' : employees, 'hq' : hq, 'old' : old}

df1 = pd.DataFrame(d)

In [21]:
df1.head()

,names,ratings,reviews,ctype,employees,hq,old


In [22]:
df1

,names,ratings,reviews,ctype,employees,hq,old


In [23]:
final_df = pd.DataFrame()

for j in range(1, 11):
    url = f"https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page={j}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    webpage = requests.get(url, headers = headers).text
    
    soup = BeautifulSoup(webpage, 'lxml')
    
    names = []
    ratings = []
    reviews = []

    ctype = []
    employees = []
    hq = []
    old = []


    for i in company:
        names.append(i.find('h2', class_ = 'companyCardWrapper__companyName').text.strip())
        ratings.append(i.find('span', class_ = 'companyCardWrapper__companyRatingValue').text.strip())
        reviews.append(i.find('span', class_ = 'companyCardWrapper__ActionCount').text.strip())
        k = i.find('span', class_ = 'companyCardWrapper__interLinking').text.strip().split(' | ')
        for j in range(len(k)):
            ctype.append(k[j])
            employees.append(k[j + 1])
            hq.append(k[j + 2])
            old.append(k[j + 3])
            break
    
    d = {'names' : names, 'ratings' : ratings, 'reviews' : reviews, 'ctype' : ctype, 'employees' : employees, 'hq' : hq, 'old' : old}

    df = pd.DataFrame(d)
    
    final_df = pd.concat([final_df, df], axis = 0, ignore_index = True)


In [24]:
final_df

,names,ratings,reviews,ctype,employees,hq,old


In [25]:
final_df.to_csv('Jobs List with details.csv', index = False)